In [17]:
from esm.pretrained import ESM3_structure_encoder_v0, ESM3_structure_decoder_v0
import torch
import numpy as np

import sys
sys.path.append('/burg/home/jb5005')
import os
import pytorch_lightning as pl

from struct_diff.data.flow import interpolate
from struct_diff.data.datamodule import ContinuousStructTokenDataModule
from struct_diff.model.models import TransformerModel
from struct_diff.model.layers import TransformerLayer, positional_encoding, time_encoding

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [40]:
pos_encoding= positional_encoding(300,1280, scale = 0.005)

In [27]:
time_enc = time_encoding(1.0, 300, 1280, scale = 0.005)
time_enc.max()

tensor(0.0050)

In [2]:
decoder = ESM3_structure_decoder_v0(device=device)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vocab = decoder.embed(torch.arange(0, 4100, device=device))

Fetching 22 files:   0%|          | 0/22 [00:00<?, ?it/s]

/pmglocal/jb5005/mambaforge/envs/struct_diff/lib/python3.12/site-packages/esm/pretrained.py:43: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(


In [3]:
vocab = decoder.embed(torch.arange(0, 4100, device=device))

In [4]:
test_embed = vocab[0,:].unsqueeze(0)
torch.argmin(torch.linalg.norm(vocab-test_embed, dim = -1))

tensor(0)

In [5]:
test_output = torch.randn([10,1280])
test_output

tensor([[-4.6240e-01,  1.2386e+00,  3.2930e-01,  ..., -2.1146e-01,
          6.7654e-01, -3.8436e-01],
        [-3.4417e-01, -1.8243e-01,  3.5289e-01,  ...,  2.7764e+00,
          1.1923e-01, -7.9778e-01],
        [ 1.3175e+00, -7.6472e-01,  1.4671e+00,  ..., -5.7807e-01,
         -2.0455e+00, -1.2015e+00],
        ...,
        [-1.5041e+00,  1.9361e+00,  1.1369e+00,  ...,  2.8015e-01,
          5.5996e-01,  2.5161e-02],
        [-9.4698e-03, -8.3337e-01,  1.3307e-05,  ...,  1.3011e-01,
          7.4232e-01, -7.0243e-01],
        [ 3.6010e-01,  1.1307e-01,  1.1625e-01,  ..., -2.0104e-01,
         -1.4844e+00, -9.4192e-01]])

In [6]:
test_embed = vocab[0:10,:]

test_stack = torch.stack([test_embed, test_embed], dim=0)
distances = torch.cdist(test_embed, vocab, p=2)

# Get the index of the minimum distance (closest row) for each input row
closest_indices = distances.argmin(dim=-1)

# Convert to a list
closest_indices.tolist()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [7]:
def convert_continuous_to_discrete_token(x0):
    """
    Maps backwards from the continuous token embedding to the discrete version
    Computes the Eucledian distance between the continuous token and all discrete tokens, 
    then selects discrete with the smallest distance

    args: x0 [BxLx1280]
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    decoder = ESM3_structure_decoder_v0(device=device)
    vocab = decoder.embed(torch.arange(0, 4100, device=device))
    distances = torch.cdist(x0, vocab, p=2)
    closest_indices = distances.argmin(dim=-1)
    return closest_indices    

In [5]:
model = TransformerModel()
layer = TransformerLayer()

In [12]:
x_orig == 0.0

tensor([[[False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         ...,
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False]]])

In [11]:
x_orig = torch.randn([1,128,1280])
x = x_orig
num_steps = 100
# model.load_from_checkpoint()
model.eval()
for i in range(num_steps):
    t = i/num_steps
    x = model(x, t)

x

tensor([[[ 0.0394,  1.5524, -1.7389,  ...,  1.7119, -0.5969,  1.0694],
         [ 0.7181,  1.0499, -1.0482,  ...,  1.7932, -0.6871,  0.9278],
         [ 0.7626,  0.0639, -0.9469,  ...,  1.8666, -0.7137,  0.7816],
         ...,
         [-0.3231,  1.5246, -1.8775,  ...,  1.8185,  0.2351,  1.3290],
         [ 0.5645,  1.5603, -2.1033,  ...,  1.8607,  0.2733,  1.3662],
         [ 1.1641,  0.8868, -1.5779,  ...,  1.8802,  0.2765,  1.3763]]],
       grad_fn=<NativeLayerNormBackward0>)

In [24]:
x_temp = x_orig

In [34]:
x_temp = model(x_temp, 1.0)
x_temp

tensor([[[ 0.0054,  1.4817, -1.2321,  ..., -0.6667,  1.8546, -0.3587],
         [-0.9143,  0.9169, -1.2277,  ..., -0.4016,  1.8338, -1.2030],
         [-0.6997,  1.7334, -0.2508,  ...,  0.0689,  0.4850, -0.4182],
         ...,
         [-0.3185,  2.1321, -0.2828,  ..., -0.5648,  1.2682, -0.5033],
         [ 0.3660,  1.8188,  0.9322,  ..., -1.3208,  2.0284, -1.5418],
         [-0.8983,  1.7011,  0.6608,  ..., -0.4690,  2.5484, -0.4190]]],
       grad_fn=<NativeLayerNormBackward0>)

In [29]:
test_output = torch.randn([60,300,1280])
convert_continuous_to_discrete_token(test_output)


/pmglocal/jb5005/mambaforge/envs/struct_diff/lib/python3.12/site-packages/esm/pretrained.py:43: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(


KeyboardInterrupt: 